In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA,SparsePCA
import numpy as np
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from scipy.stats import uniform, truncnorm, randint
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.utils import class_weight
import gensim
from gensim.models import KeyedVectors, Word2Vec, FastText
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix,classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import ComplementNB

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/python-for-ml/sent-analysis/master/finaldataset.csv')


In [3]:
#data = pd.read_csv("finaldataset.csv")
data=data.drop(['title_x','title_y','title','value'],axis=1)
data.head()

,data,tag
0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন,কিছুটা নেতিবাচক
1,এটা কেন হচ্ছে সংশ্লিষ্ট সকলের ডিপ্রেশনের ফলে ন...,কিছুটা নেতিবাচক
2,আমাদের দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে ব...,নিশ্চিত নেতিবাচক
3,চুরি নয় লুটপাট,নিশ্চিত নেতিবাচক
4,ইসলামী ব্যাংকের বর্তমান অবস্থা দেখে মনে হয় শাস...,নিরপেক্ষ


In [4]:
le = LabelEncoder()
data['tag'] = le.fit_transform(data['tag'])
data.head()

,data,tag
0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন,1
1,এটা কেন হচ্ছে সংশ্লিষ্ট সকলের ডিপ্রেশনের ফলে ন...,1
2,আমাদের দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে ব...,4
3,চুরি নয় লুটপাট,4
4,ইসলামী ব্যাংকের বর্তমান অবস্থা দেখে মনে হয় শাস...,2


In [5]:
data['tag']=data['tag'].replace(3, 0)
data['tag']=data['tag'].replace(4, 1)
data = data[data['tag'] != 2]

In [6]:
d = data['data'].tolist()
label = data['tag'].tolist()

In [7]:
for i in range(len(d)):
    d[i] = re.sub(r'[^\w\s]', '', d[i])

In [8]:
corpus = []
for i in range(len(d)):
    corpus.append(word_tokenize(d[i]))

In [9]:
vectorizer=CountVectorizer()
vocabulary=vectorizer.fit(d)
X= vectorizer.transform(d)
size = round(len(vocabulary.get_feature_names())**0.65)

In [10]:
size

651

In [11]:
wvmodel = Word2Vec(corpus, size=size, window=4, min_count=1)

In [12]:
X = np.zeros((len(corpus),size)) #Initializing the X matrix with zeros
for i in range(len(corpus)):
    emb = np.zeros((1,size))
    for w in corpus[i]:
        emb = emb +  wvmodel[w]
    X[i] = emb 

<ipython-input-12-35b75f0a5797>:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  emb = emb +  wvmodel[w]


In [13]:
X_link = 'https://raw.githubusercontent.com/python-for-ml/sent-analysis/master/X.csv'
X = pd.read_csv(X_link, header = None).to_numpy()

label_link = 'https://raw.githubusercontent.com/python-for-ml/sent-analysis/master/label.csv'
label = pd.read_csv(label_link, header = None)
label = label.to_numpy().reshape([1, len(label)])[0]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,label, test_size=0.2)

In [15]:
Model = GaussianNB()
Model.fit(X_train, y_train)

GaussianNB()

In [16]:
y_pred = Model.predict(X_test)

In [17]:
print("Accuracy  :",metrics.accuracy_score(y_test, y_pred))
print("Precision :",metrics.precision_score(y_test,y_pred))
print("F1-Score  :",metrics.f1_score(y_test,y_pred))
print("Recall    :",metrics.recall_score(y_test,y_pred))

Accuracy  : 0.7015200368493781
Precision : 0.7493333333333333
F1-Score  : 0.7762430939226518
Recall    : 0.8051575931232091


In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.59      0.51      0.55       775
         1.0       0.75      0.81      0.78      1396

    accuracy                           0.70      2171
   macro avg       0.67      0.66      0.66      2171
weighted avg       0.69      0.70      0.70      2171



In [19]:
confusion_matrix(y_test,y_pred)

array([[ 399,  376],
       [ 272, 1124]], dtype=int64)